<img src="https://github.com/jupytercon/2020-exactlyallan/raw/master/images/RAPIDS-header-graphic.png">

# Explanatory Data Visualization
***Interactive presentation dashboards***


In [ ]:
## Run this cell to show the next section's walkthrough video ##
from IPython.display import HTML
HTML('<iframe width="560" height="315" src="https://www.youtube.com/embed/GJnUGqYj7D0" frameborder="0" allow="accelerometer; autoplay; clipboard-write; encrypted-media; gyroscope; picture-in-picture" allowfullscreen></iframe>')


## Overview
This final notebook is geared towards taking the previous findings and preparing them for presentation through an interactive Plotly Dash visualization application powered by cuDF and cuGraph's PageRank. 

### cuDF, cuGraph, cuxfilter, and Plotly Dash
- [cuDF](https://docs.rapids.ai/api/cudf/stable/) is a RAPIDS GPU DataFrame library for manipulating data with a pandas-like API.

- [cuGraph](https://docs.rapids.ai/api/cugraph/stable/) is a RAPIDS library for GPU accelerated graph analytics with functionality like NetworkX.

- [cuxfilter](https://docs.rapids.ai/api/cuxfilter/nightly/) is a RAPIDS visualization library for cross-filtering data, designed to quickly build linked dashboards powered by cuDF compute capabilities. 

- [Plotly Dash](https://plotly.com/dash/) is a framework for specifying production ready visualization applications all in Python. 

## Dashboard Concepts and Audiences
We've taken a bike share dataset, explored it, run analytics on it, and now have confidence in our ability to highlight the most important stations for two key usage patterns. The next step is communicating our findings - something at which visualization excels.

However, a viz needs to be appropriate for the data it is showing, the audience it is intended to be shown to, and the medium or context it will be shown.

For instance, is the presentation to highly technical colleagues already familiar with your work, as you drive the presentation from your personal machine? Or to executives at a board room? Or completely asynchronous through a web site with a wide range of audience expertise levels?

Thinking about these ahead of time and preparing for them will lead to more successfully communicating your findings.

As you think of this, it helps to explore previous works, such as the [Plotly Dash Gallery](https://dash-gallery.plotly.host/Portal/), for ideas and to see best practices (or worst practices, I'm looking at you 3D pie chart...).



In [ ]:
## Run this cell to show the next section's walkthrough video ##
from IPython.display import HTML
HTML('<iframe width="560" height="315" src="https://www.youtube.com/embed/JyZs4ApG7yI" frameborder="0" allow="accelerometer; autoplay; clipboard-write; encrypted-media; gyroscope; picture-in-picture" allowfullscreen></iframe>')
     

### A Note About Sketching and Design Iterations
<img src="https://raw.githubusercontent.com/jupytercon/2020-exactlyallan/master/images/dashboard-sketch-ideas.jpg" />
<img src="https://raw.githubusercontent.com/jupytercon/2020-exactlyallan/master/images/plotly_dashboard_sketch.jpg" />

A well designed visualization owes as much to iteration as it does to skill (though experience helps). The more iterations, generally the better the viz. However, the mental overhead of our technical tools often get in the way of our thought process.

By the time it takes to create a new notebook cell, look up the syntax for your favorite viz library, and load data - you could have quickly generated several iterations of ideas through sketches. 

As shown from our sketches above, they don't have to be high fidelity or even good - just enough to try out ideas quickly and communicate to colleagues. 

### Try It Now
Pull out a piece of paper and do a few thumbnail sized sketches of variations on this dashboard - spending no more than 5-10 minutes. The messier the better. 

Thumbnail sketches are for thinking and personal consumption. Larger ones come later to help communicate ideas to colleagues. 

## Imports
Now that we have sketched out our idea, lets prototype them. As usual, make sure the necessary imports are present to load, as well as setting the data location.

In [ ]:
## Run this cell to show the next section's walkthrough video ##
from IPython.display import HTML
HTML('<iframe width="560" height="315" src="https://www.youtube.com/embed/R5AbKqo2Uvk" frameborder="0" allow="accelerometer; autoplay; clipboard-write; encrypted-media; gyroscope; picture-in-picture" allowfullscreen></iframe>')


In [ ]:
import cudf
import plotly.graph_objects as go
import plotly.express as px
from jupyter_dash import JupyterDash
import dash_core_components as dcc
import dash_html_components as html
from dash.dependencies import Input, Output
import pandas as pd
import cugraph
import cuxfilter
from pathlib import Path

DATA_DIR = Path("../data")
FILENAME = Path("modified_trips.parquet")


In [ ]:
trips = cudf.read_parquet(DATA_DIR / FILENAME)

In [ ]:
trips['time_of_day'] = 0 # day
trips.loc[trips.query('hour>19 or hour<8').index, 'time_of_day'] = 1 # night

In [ ]:
# create a day_type string map
day_type_map = {0:'weekday', 1:'weekend', '':'all'}
time_of_day_map = {0:'day(8am-8pm)', 1:'night(8pm-8am)', '':'all'}

### Dashboard Mockup with cuxfilter
With our sketch idea of what we want the final explanatory visualization to look like, and what data it will show, lets try and create an interactive mockup to test our concept.

As usual, we will load the data into cuDF and spec out the cuxfilter chart:

In [ ]:
cux_df = cuxfilter.DataFrame.from_dataframe(trips)

In [ ]:
# Specifying a scatter plot chart will use Datashader and its required parameters
charts = [
    cuxfilter.charts.scatter(x='x', y='y', tile_provider='CARTODBPOSITRON',
                           point_size=3, pixel_shade_type='linear', pixel_spread='spread',
                          title='All Trips'),
    cuxfilter.charts.bar('all_time_week', title='Rides per week'),
    cuxfilter.charts.multi_select('day_type', label_map=day_type_map),
    cuxfilter.charts.multi_select('hour'),
]

# Generate the dashboard, select a layout and theme
d = cux_df.dashboard(charts, layout=cuxfilter.layouts.feature_and_base, theme=cuxfilter.themes.rapids)


In [ ]:
# IMPORTANT: replace notebook_url with your jupyterhub/binder base url
# IMPORTANT: if your notebook environment is in jupyterhub, set service_proxy='jupyterhub', otherwise set to 'none'
BASE_URL = 'http://localhost:8888/'
d.show(notebook_url=BASE_URL, service_proxy='none')

## Mockup Results
The cuxfilter mockup should look something like this:
<img src="https://raw.githubusercontent.com/jupytercon/2020-exactlyallan/master/images/notebook_04_dashboard_1.png" />

Overall the design seems to work, with the obvious caveat that we have yet to see PageRank in action. Nevertheless, because of how quick it is to build an interactive dashboard with cuxfilter, it can work well as a mock up tool.

If your design calls for chart types or features which are difficult to fully test (like arbitrary function calls), mocking up still makes sense for even component elements or interactions, supplemented with tools like hvplot or Datashader.

Mock ups are particularly important if you haven't connected real or a full set of your data to a visualization yet, since building an explanatory / production level application takes substantial effort (even with a simple API). Therefore, skipping lower fidelity interactive mockups will almost certainly end up wasting time on rework. With data visualization there are always surprises from unuseable results, slow performance, or the limitations of various chart interactions.

As mentioned above with sketching, increasing the number of design iterations your visualization goes through improves its quality, and interactive mockups are a useful tool for that purpose.



## Production Ready <br> Plotly Dash Visualization Application <br> with Real-time Page Rank Compute
Now that we are confident that our chart types and interactions are appropriate, lets build the dashboard with help from the [Plotly Dash Documentation](https://dash.plotly.com/).

First lets load the data into a cuDF and prepare it for the vis:

In [ ]:
## Run this cell to show the next section's walkthrough video ##
from IPython.display import HTML
HTML('<iframe width="560" height="315" src="https://www.youtube.com/embed/nOrVlzvT5Eg" frameborder="0" allow="accelerometer; autoplay; clipboard-write; encrypted-media; gyroscope; picture-in-picture" allowfullscreen></iframe>')


In [ ]:
# Load the stations data from first notebook
stations = cudf.read_csv(DATA_DIR / "stations.csv")

# Get station names
station_names = trips[['from_station_id', 'from_station_name']].drop_duplicates()
station_names.columns = ['station_id', 'station_name']

# Get total trips per station
total_trips = (trips.groupby('from_station_id').size() + trips.groupby('to_station_id').size()).reset_index()
total_trips.columns = ['station_id', 'total_trips']

# Add total trips to dataframe
stations = stations.merge(total_trips, on='station_id')
stations = stations.merge(station_names, on='station_id')

In [ ]:
stations.head()

## Define Application Layout and Style
Plotly Dash apps use standard web elements to define layouts and styles through `html`, `styles`, `class` and `css`. You can learn more on the [Dash layouts documentation](https://dash.plotly.com/layout). For our example we are using a locally hosted `css` file in the default folder: `/assets/dash-style.css` and inline `sytles.` 

Here we define our app and the layout to have a title `h1` tag, a `div` side bar for total trips and two drop down menus, and another `div` to contain the map chart and bar chart below: 

In [ ]:
app = JupyterDash(__name__)

app.layout = html.Div([
    html.Div([
        html.H3(["Divvy Bikeshare Chicago"]),
        html.H5(["Total Selected Trips:"]),
        dcc.Loading(
            dcc.Graph(id = 'number', figure = go.Figure(go.Indicator(mode = "number", value = trips.shape[0])),
            style = {
            'height': '250px'
            }),
            color = '#b0bec5'
        ),
        html.H5(["Day of Week:"]),
        dcc.Dropdown(id = 'day', clearable = False, value = '',
            options = [{'label': day_type_map[c],'value': c} for c in day_type_map]
        ),
        html.H5(["Time of Day:"]),
        dcc.Dropdown(id = 'time', clearable = False, value = '',
            options = [{'label': time_of_day_map[c], 'value': c} for c in time_of_day_map]
        )],
        style = {
            'z-index' : '99',
            'position': 'absolute',
            'width': '15%',
            'height': 'calc(100% - 2em)',
            'padding': '1em 2em',
            'background-color': '#aabacc',
            'color': 'rgb(70, 105, 130)',
            'box-shadow': '5px 0px 3px 0px rgba(0,0,0,0.1)'
        }
    ),
    html.Div([
        html.Div([
            html.H5(["Station Importance PageRank(Color) by Trips(Size)"]),
            dcc.Graph(id = 'pagerank_plot',
                config = {'responsive': True, 'modeBarButtonsToRemove': ['select2d', 'lasso2d']}
            )
        ],
        style = {
            'display': 'inline-block',
            'width': '100%',
            'vertical-align': 'top'
        }),
        html.Div([
            html.H5(["Total Trips Per Week (2014-2017)"]),
            dcc.Graph(id = 'all_time_week_bar',
                config = {'responsive': True, 'modeBarButtonsToRemove': ['zoom2d', 'zoomIn2d', 'zoomOut2d']}
            )
        ],
        style = {
            'display': 'inline-block',
            'width': '100%'
        })
    ],
    style = {
        'width': 'calc(80% - 6em)',
        'height': 'auto',
        'margin-left': 'calc(15% + 6em)',
        'padding-top': '2em',
        'display': 'inline-block',
        'vertical-align': 'top',
        'color': '#aabacc'
    })
],
style = {
    'position': 'relative',
    'border-bottom': '2px solid #aabacc'
})


## Define Function to Generate Plots with Plotly Express
Next lets define the functions to build our two charts and link them to our data:

In [ ]:
# Geospatial bubble chart based on Page Rank and Trip data
def get_pagerank_plot(data):
    df = calculate_page_rank(data).to_pandas()
    g = px.scatter_mapbox(df, lat="lat", lon="lon", color="pagerank", size="total_trips",
                          hover_data=["station_name"], mapbox_style="carto-positron",
                          color_continuous_scale=px.colors.cyclical.Edge_r, size_max=15, zoom=10,
                          height=700
                         )
    g.layout['uirevision'] = True
    return g

# Bar chart based on total trips over weeks
def get_week_bar_chart(data):
    all_time_week_df = data.groupby('all_time_week').size().reset_index()
    all_time_week_df.columns = ['week', 'trips']
    g = px.bar(all_time_week_df.to_pandas(), 
               x="week", y='trips', template=dict(layout={'selectdirection': 'h',}), 
               height=300
              )
    g.layout['dragmode']='select'
    g.layout['uirevision'] = True
    return g

## Define Function to Calculate Page Rank
Because Plotly Dash applications are hosted through a python backend, the web based charts are able to call custom python functions. Lets use this feature and the speed of cuGraph to calculate new PageRank scores base on a user's selection:

In [ ]:
def calculate_page_rank(data):
    G = cugraph.Graph()
    G.from_cudf_edgelist(data, source='from_station_id', destination='to_station_id')
    data_page = cugraph.pagerank(G)
    return data_page.merge(stations, left_on='vertex', right_on='station_id').drop(columns=['vertex'])

## Define Events and Callbacks
Here we define what happens when a user interacts with chart selections through [Dash callbacks](https://dash.plotly.com/basic-callbacks):

In [ ]:
def bar_selection_to_query(selection, column):
    """
    Compute pandas query expression string for selection callback data
    Args:
        selection: selectedData dictionary from Dash callback on a bar trace
        column: Name of the column that the selected bar chart is based on
    Returns:
        String containing a query expression compatible with DataFrame.query. This
        expression will filter the input DataFrame to contain only those rows that
        are contained in the selection.
    """
    point_inds = [p['label'] for p in selection['points']]
    xmin = min(point_inds)  # bin_edges[min(point_inds)]
    xmax = max(point_inds) + 1  # bin_edges[max(point_inds) + 1]
    xmin_op = "<="
    xmax_op = "<="
    return f"{xmin} {xmin_op} {column} and {column} {xmax_op} {xmax}"

# Define callback to update graph, id ties plot code to layout
@app.callback(
    [
        Output('pagerank_plot', 'figure'),
        Output('all_time_week_bar', 'figure'),
        Output('number', 'figure')
    ],
    [
        Input("day", "value"), Input("time", "value"),
        Input("all_time_week_bar", "selectedData")
    ]
)
def update_figure(day, time, selected_weeks):
    query = ['day_type == '+str(day) if day != "" else "", 'time_of_day =='+str(time) if time != "" else ""]
    query_str = ' and '.join([x for x in query if x != ""])
    
    data = trips
    if len(query_str) > 0:
        data = trips.query(query_str)

    week_bar_chart = get_week_bar_chart(data)
    
    if selected_weeks is not None:
        query.append(bar_selection_to_query(selected_weeks, 'all_time_week'))
        query_str = ' and '.join([x for x in query if x != ""])
        if len(query) > 0:
            data = trips.query(query_str)
    
    pagerank_plot = get_pagerank_plot(data)
    
    number = go.Figure(go.Indicator(
                mode="number",
                value=data.shape[0]
            ))

    return pagerank_plot, week_bar_chart, number

## Start the Plotly Dash Visualization
Now that we have defined everything, lets run the application:

In [ ]:
# NOTE: If you are running in a JupyterHub environment, run the below command:
# JupyterDash.infer_jupyter_proxy_config()

# NOTE: For Jupyterlab run: 
# app.run_server(mode="jupyterlab")

# NOTE: To run inline with a notebook (NOT recommended): 
# app.run_server(mode="inline")

# NOTE: To run as seperate tab run then click on the link (recommended):
app.run_server(debug=False)


## Final Plotly Dash Visualization
This is what the dashboard should look like:

<img src="https://raw.githubusercontent.com/jupytercon/2020-exactlyallan/master/images/PlotlyDash-Dashboard.png">
          
Overall the speed of cuGraph's PageRank as well as the simple interactions make this dashboard intuitive and usable for quickly finding the most important bike stations. With preset filters, it succinctly gives an overview of how this complicated network behaves over time and is able to handle new data seamlessly (not bad for a tutorial app).  

### Try It Now
See if you can adjust the layout of the above app by reordering the `div` tags and changing the `style` tag values.

Or try changing the `css` by adding a reference to `external_stylesheets` as shown below. You can use the external `css` files from example GitHub repos from their [Dash Gallery](https://dash-gallery.plotly.host/Portal/). 


```
external_stylesheets = ['https://codepen.io/chriddyp/pen/bWLwgP.css']

app = dash.Dash(__name__, external_stylesheets=external_stylesheets)
```

NOTE: you'll have to re-run all of the Plotly related cells after updating. Be forewarned, editing css in a notebook is a lot like [this](http://gph.is/1heneJM).

## A Final Summary on the Benefits of <br> Running with RAPIDS

Hopefully as you've clicked through these tutorial notebooks, you've noticed how seamless it is working within the RAPIDS libraries and with other libraries. One of the key goals of RAPIDS is to keep the tools and workflows you are familiar with, but turn them into end-to-end GPU accelerated pipelines. From ETL, exploration, analytics, and visualization - you can take advantage of the speed ups from GPUs.

We on the viz team are continuing to integrate with other visualization libraries, and have projects in the works to improve the performance and capabilities of web visualizations even further.

RAPIDS is still a relatively young project (we aren't even to 1.0 yet!), but we continue to work towards building out more features and improving. Stay up to date with our projects on our [Home](https://rapids.ai/), [GitHub](https://github.com/rapidsai), and [Twitter page](https://twitter.com/rapidsai).

In [ ]:
## Run this cell to show this section's walkthrough video ##
from IPython.display import HTML
HTML('<iframe width="560" height="315" src="https://www.youtube.com/embed/deGQdljxYlY" frameborder="0" allow="accelerometer; autoplay; clipboard-write; encrypted-media; gyroscope; picture-in-picture" allowfullscreen></iframe>')
